### Data Preprocessing Module

This module includes the class `DataPrep` and the functions that go inside it.
Each function will help with specific tasks in data preprocessing and getting the data ready for feature engineering 
and training the model. 

We are going to use resources that will help us with data manipulation. This includes libraries such as NumPy and Pandas.


There will be specific description for each function and their usage.

List of functions in this file:


In [1]:
import numpy as np
import pandas as pd
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

c:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.25.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


### Extract Date
This function extracts the year, month, day, and quarter from the `Date` column in the DataFrame that is passed to it as a parameter. 

The properties related to date are added in order for better readability of the DataFrame and columns.

In [11]:
class DataPrep:
    
    def __init__(self, df):
        self.df = df

    def extract_date(self):
        """
        Extract the date from the DataFrame and merge the columns back into the DataFrame

        Parameters: 
            self.df (DataFrame): DataFrame containing the stock price data.

        Parameter constraints:
            Should have a column 'date'
            The format of the date should be valid

        Returns:
            DataFrame: The modified DataFrame with 'year', 'month', 'day, and 'quarter' added as new columns
        """
        # Setting the Date column to Panda's datetime format
        self.df['Date'] = pd.to_datetime(self.df['Date'])

        #Extracting the year and adding it after 'Date' column
        self.df['Year'] = self.df['Date'].dt.year
        index_date = self.df.columns.get_loc('Date')
        self.df.insert(index_date + 1, 'Year', self.df.pop('Year'))

        # Extracting the month and adding it after 'Year' column
        self.df['Month'] = self.df['Date'].dt.month
        index_year = self.df.columns.get_loc('Year')
        self.df.insert(index_year + 1, 'Month', self.df.pop('Month'))

        # Extracting the day and adding it after 'Month' column
        self.df['Day of Week'] = self.df['Date'].dt.dayofweek
        index_month = self.df.columns.get_loc('Month')
        self.df.insert(index_month + 1, 'Day of Week', self.df.pop('Day of Week'))

        # Extracting the quarter and adding it after 'Day' column
        self.df['Quarter'] = self.df['Date'].dt.quarter
        index_day = self.df.columns.get_loc('Day of Week')
        self.df.insert(index_day + 1, 'Quarter', self.df.pop('Quarter'))

        return self.df
    
    def handle_missing_data(self):
        """
        This function will check for any null values in our dataset and handles it with sklearn's iterative imputation

        Parameteres:
            self.df (DataFrame): DataFrame containing the stock price data.

        Returns:
            The modified DataFrame with all the missing values handled.
        """
        # Taking the columns containing missing values
        columns_with_missing_value = self.df.columns[self.df.isnull().any()].tolist()

        # Initiating an iterative imputer object as imputer
        imputer = IterativeImputer(max_iter=10, random_state=42)

        # Fitting the columns with missing values to the imputer
        imputer.fit(self.df[columns_with_missing_value])

        # Get the imputed data from the imputer and transforimg the columns with the imputed data
        imputed_data = imputer.fit_transform(self.df[columns_with_missing_value])

        # Replacing the columns with missing values with the columns containing imputed data
        self.df[columns_with_missing_value] = imputed_data

        return self.df

### _The code blocks below is for testing the class and its functions_


In [8]:
df = pd.DataFrame({'Date' : ['2023-09-15', '2023-09-16', '2023-09-17'],
                   'High' : [21,22,23],
                   'Low' : [15,16,17],
                   'Volume' : [1200,2300,2400],
                   'Close' : [22,26,29]
                   })

df0 = DataPrep(df).extract_date()
df0


,Date,Year,Month,Day of Week,Quarter,High,Low,Volume,Close
0,2023-09-15,2023,9,4,3,21,15,1200,22
1,2023-09-16,2023,9,5,3,22,16,2300,26
2,2023-09-17,2023,9,6,3,23,17,2400,29


In [9]:

data = pd.DataFrame({
    'Age': [25, 30, np.nan, 22, 28],
    'Income': [45000, 60000, np.nan, 35000, np.nan],
})

data0 = DataPrep(data)
data0.handle_missing_data()

,Age,Income
0,25.00,45000.000000
1,30.00,60000.000000
2,26.25,46666.667486
3,22.00,35000.000000
4,28.00,46666.669946
